In [126]:
# coding: utf-8
"""
tf-idfによるトピックの抽出
"""
from gensim import corpora, models, similarities
import csv
import MeCab
import numpy as np
from sklearn import cluster
import collections

# csvファイルの読み込み
def readcsv(path):
    f = open(path, "rb")
    dataReader = csv.reader(f)
    arr = [row for row in dataReader]
    return arr

def writecsv(arr, path):
    f = open(path, "ab")
    dataWriter = csv.writer(f)
    dataWriter.writerows(arr)
    f.close()
    
def parsing(sentence):
    mecab = MeCab.Tagger("-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd")
    res = mecab.parseToNode(sentence)
    list_words = []
    while res:
        features = res.feature.split(",")
        if (features[0] == "名詞" and features[1] in ["一般", "固有名詞", "サ変接続", "形容動詞語幹"]) or features[0] == "形容詞":
            if features[6] == "*":
                list_words.append(res.surface)
            else:
                list_words.append(features[6])
        res = res.next
    return list_words

# 正規化する関数
def normalize(vector):
    tmp = np.sqrt(np.sum(vector * vector))
    if tmp == 0:
        return vector
    else:
        return vector / tmp

形態素解析して、bag of words を作成

In [112]:
list_corpus = readcsv("./files/file_1.csv")
list_corpuses = [row[1].split("。") for row in list_corpus]
list_corpuses = [sentence for row in list_corpuses for sentence in row]
list_bag_of_words = [parsing(sentence) for sentence in list_corpuses]

辞書をダウンロードして、コーパスにidを振る

In [113]:
dictionary = corpora.Dictionary.load_from_text("./files/dictionary1.txt")
corpus = [dictionary.doc2bow(text) for text in list_bag_of_words]

tfidfモデルを作成して、計算、スパースマトリックスを変換する

In [115]:
tfidf = models.TfidfModel(corpus)
tfidf_corpus = [tfidf[corpus[i]] for i in range(len(corpus))]
tfidf_corpus_rev = np.zeros((3224, 1109))
for i, row in enumerate(tfidf_corpus):
    for j in row:
        tfidf_corpus_rev[i][j[0]] = j[1]
tfidf_corpus_rev = [normalize(row) for row in tfidf_corpus_rev]

階層クラスタリングにより、tf-idfをクラスタリング

In [117]:
hieral = cluster.AgglomerativeClustering(n_clusters=13, affinity='euclidean', linkage='ward')
result = hieral.fit_predict(tfidf_corpus_rev)

作成したクラスタの中でbag-of-wordsを作成する

In [119]:
dict_cluster_word = collections.defaultdict(list)
for i, num in enumerate(result):
    for word in list_bag_of_words[i]:
        dict_cluster_word[num].append(word)

クラスタ内の

In [123]:
num = 13
list_word = sorted(collections.Counter(dict_cluster_word[num]).items(),
                   key=lambda x: x[1], reverse=True)
for word in list_word:
    print word[0]

施設
居室
広い
スペース
個室
建物
清潔
良い
部屋
綺麗
共有
バリアフリー
やすい
入居
きれい
ホテル
新しい
ない
場所
ロビー
清潔感
プライバシー
生活
自立
移動
掃除
家具
清掃
中庭
食事
フロント
明るい
日当たり
外観
スタッフ
車椅子
車いす
手すり
内装
トイレ
感じ
確保
安心
室内
共用
雰囲気
段差
高級
きれいな
教育
対応
人
ソファ
快適
マンション
駐車場
多い
印象
食堂
自体
眺め
ラウンジ
見学
空間
基本的
家族
車
歓談
豪華
室
満足
玄関
無い
窓
手入れ
よい
臭い
最高
嬉しい
環境
高い
素晴らしい
介護
設置
使用
丁寧
入り口
おしゃれ
方々
全館
心
月
十分
素敵
防音
訪問
充実
上品
隅々
ドア
面
大きい
駅
大変
完全
イベント
廊下
フロアー
お洒落
不審者
高層
共用設備
サービス
ホテルマン
景色
持ち込み
希望
高齢者
洋風
良好
館内
易い
お見舞い
狭い
便利
感心
開放的
対策
保護
目
静か
インテリア
南
完備
元気
リゾートホテル
ヨーロッパ
優雅
友人
調度
要介護
風呂
なかった
庭
プライベート
職員
タイプ
活き活き
中
ポイント
訪問者
個所
作り
居住空間
気
レンガ
自由
入浴
気持ち
天井
自分
状態
の・ようなもの
レストラン
造り
設備
フラット
通路
買い物
普通
モチーフ
バリアーフリー
カビ
最低
レジャー施設
心配
庭園
高台
内部
美しい
資産
不便
関連会社
棟
専用
行動
窓口
好き
つくり
充分
好み
縁
状況
経過
開業
エントランス
鍵
好感
個別
壁
衣料品
がらん
遜色
スペ
夜景
確か
形状
来訪
採用
老人ホーム
住宅地
最低限
クロゼット
2013年
自然
芝生
物件
生活レベル
提供
女性
味
面積
クオリティ
美味しい
屋上
アウト
パート
点検
一流
見た目
10月
シアター
自主
友達
利用
一致
恐縮
空気
良質
非常
様子
如何
安らぎ
あかるい
図書館
最寄駅
子供
公園
ゴージャス
各所
荻窪
いい
盛り
物
談笑
悪い
ベッド
週
特筆
旅館
サポート
経堂駅
バス
屋外
難点
重厚
回廊
娯楽施設
配慮
整備
圧倒
共同浴場
手摺
キッチン
定期的
気兼ね
ユニットケア
完璧
前面
モットー
他
魚